In [1]:
cd ..

/home/dmitriishubin/Desktop/physionet-challenge-2020


In [2]:
#### insert the name of the last run
!tensorboard --logdir=runs/May11_01-21-42_dmitrii

/home/dmitriishubin/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dmitriishubin/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dmitriishubin/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dmitriishubin/anaco

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import gc
import ast
import json
import os

from metrics import Metric
from utils.KPI_plots import plot_confusion_matrix

In [3]:
%matplotlib qt

# Settings

In [4]:
competition_metric = Metric()

# Dataset A

In [5]:
DEBUG_PATH = './data/CV_debug/A/'
DATA_PATH = './data/A/formatted/'

list_records = [i[:-5] for i in os.listdir(DEBUG_PATH) if i.find('.json')!=-1]


In [6]:
def load_waveform(data_path,name):
    
    signal = np.load(data_path+name+'.npy')
        
    return signal

def load_label(data_path,name):
    
    label = json.load(open(data_path+name+'.json'))
        
    return label

In [7]:
# Main processing pipeline
scores_errors = np.array([])
scores_competition = np.array([])
records = np.array([]).astype(np.str)
labels = [] #np.array([])
preds = [] #np.array([])

#load all records
for record in list_records:
    
    label = load_label(DATA_PATH,record)
    records = np.append(records,label['filename'])
    label = label['labels_training_merged']
    
    pred = load_label(DEBUG_PATH,record)
    pred = pred['predicted_label']
    
    #calc score for each record
    label = np.array(label).reshape(1,-1)
    pred = np.array(pred).reshape(1,-1)
    
    scores_competition = np.append(scores_competition,competition_metric.compute(label, pred))
    scores_errors = np.append(scores_errors,np.sum(np.abs(label - pred)))
    
    #add predictions and labels for overall KPI estimation
    labels.append(label[0,:])#labels = np.append(labels,label,axis=0)
    preds.append(pred[0,:])#preds = np.append(preds,pred,axis=0)

    
preds = np.array(preds)
labels = np.array(labels)
#plot modified consustion matrix
matrix = competition_metric.compute_modified_confusion_matrix(labels, preds)
label_names = ['IAVB'
'AF',
'AFL',
'Brady',
'CRBBB',
'IRBBB',
'LAnFB',
'LAD',
'LBBB',
'LQRSV',
'NSIVCB',
'PR',
'PAC',
'PVC',
'LPR',
'LQT',
'QAb',
'RAD',
'RBBB',
'SA',
'SB',
'SNR',
'STach',
'SVPB',
'TAb',
'TInv',
'VP'
 ]


plot_confusion_matrix(matrix,label_names)

/home/dmitriishubin/Desktop/physionet-challenge-2020/utils/KPI_plots.py:54: RuntimeWarning: invalid value encountered in true_divide
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


In [8]:
print('Metric for dataset A: 'competition_metric.compute(labels, preds))

SyntaxError: invalid syntax (<ipython-input-8-73b21cc80c72>, line 1)

In [33]:
#keeep records with errors only
records = records[np.where(scores_errors > 0)]
scores_errors = scores_errors[np.where(scores_errors > 0)]

#TODO: clarify the type of sort
score_order = np.argsort(scores_errors)
records = records[score_order[::1]]

#save a list of files in csv
pd.DataFrame(records).to_csv('./data/dataset_A_list.csv')

In [48]:
def plot_record(record):
    
    ecg = np.load(DATA_PATH+record+'.npy')
    #ecg = preprocessing.run(ecg)
    meta = json.load(open(DATA_PATH+ f'{record}.json'))
    
    pred = load_label(DEBUG_PATH,record)
    pred = pred['predicted_label']
    
    #heatmap = np.array(data[record]['heatmap'],dtype=np.float)
    
    #plot the data
    fig,ax = plt.subplots(figsize=(20,20))
    fig.suptitle(record+', labels: '+str(meta['labels_full']))
    for i in range(12):
        ax.plot(ecg[:,i]+500*i)
    plt.show()
    
    print('Predictions: ',np.array(meta['labels_training_merged']) - np.array(pred))
    #print('Label:       ',meta['labels_training_merged'])
    return 0

plot_record(record=records[0])

Predictions:  [-7.87102508e-08 -1.06736738e-11 -1.55952414e-17 -2.33664234e-11
 -1.33120404e-10 -6.44375291e-08 -3.14574755e-11 -2.11950519e-05
 -1.76306276e-08 -1.45609177e-10 -8.04824936e-08 -3.31910644e-15
 -4.20201195e-06 -1.30721836e-13 -2.53869176e-10 -2.83426442e-11
 -3.73308584e-09 -2.26913453e-12 -1.76738264e-38 -3.29476370e-06
 -2.23708483e-08  1.31130219e-06 -1.26122125e-11 -4.67066483e-38
 -5.08948226e-08 -2.30375302e-11 -7.60597565e-38]


0